# 5.1 层和块

单个神经网络（1）接受一些输入；（2）生成相应的标量输出；（3）具有一组相关参数，更新这些参数可以优化某目标函数。

当具有多个输出的网络时，我们利用矢量化算法来描述整层神经元。像单个神经元一样，层（1）接受一组输入，（2）生成相应的输出，（3）有一组可调整的参数描述。当我们使用softmax回归时，一个单层本身就是模型。然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上述基本架构。

为了实现复杂的网络，引入块的概念。块可以描述单个层、由多个层组成的组件或整个模型本身。使用块进行抽象的好处是可以将一些块组合成更大的组件，这一过程通常是递归的。通过定义代码来按需生成任意复杂度的块，我们可以通过简洁的代码实现复杂的神经网络。

从编程的角度看，块由类（class)表示。它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，并且必须存储任何必需的参数。最后，为了计算梯度，块必须具有反向传播函数。

In [1]:
#下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层，然后是一个具有10个隐藏层且不带激活函数的全连接输出层。
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1502, -0.0148, -0.1751,  0.0648, -0.2747,  0.0635, -0.0209, -0.0325,
         -0.1510, -0.1641],
        [ 0.0020, -0.0005, -0.0854,  0.0230, -0.3277, -0.0281, -0.0748, -0.0582,
         -0.0642, -0.0763]], grad_fn=<AddmmBackward0>)

我们通过实例化nn.Sequential来构建我们的模型，层的执行顺序是作为参数传递的。简而言之，nn.Sequential定义了一种特殊的Module，即在Pytorch中表示一个块的类，它维护了一个由Module组成的有序列表。注意，两个全连接层都是Linear类的实例，Linear类本身就是Mudule的子类。
net(X)实际上是net.__call__(X)的简写。

## 5.1.1 自定义块

每个块必须的功能：
1.将输入数据作为其前向传播函数的参数；
2.通过前向传播来生成输出。请注意，输出的形状可能与输入的形状不同。
3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
4.存储和访问前向传播计算所需的参数。
5.根据需要初始化模型参数。

In [2]:
#从零开始编写一个块。它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
#MPL类继承了表示块的类。我们的实现只需要提供我们自己的构造函数和前向传播函数。

class MLP(nn.Module):
    # 用模型参数生命层。这里，我们声明两个全连接层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        # 这样，在这类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，我们这里使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

前向传播函数，它以X作为输入，计算带有激活函数的隐藏表示，并输出其未规范化的输出值。在这个MLP实现中，两个层都是实例变量。
接着实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层。
注意一些细节：1.我们定制的__init__函数通过super().__init__()调用父类的__init__函数，省去了重复编写模板代码的痛苦。
2.我们实例化两个全连接层，分为别self.hidden和self.out。
3.除非我们实现一个新的运算符，否则不必担心反向传播函数或参数初始化，系统将自动生成这些。

In [3]:
net = MLP()
net(X)

tensor([[-0.0935,  0.2152, -0.2614,  0.0323,  0.1411, -0.1798,  0.1329,  0.0625,
         -0.0867, -0.1276],
        [-0.0326,  0.1894, -0.2613, -0.1224,  0.1880, -0.1510,  0.0631,  0.1629,
         -0.1698,  0.0049]], grad_fn=<AddmmBackward0>)

块的一个主要优点是它的多功能性。我们可以子类化块以创建层、整个模型或具有中等复杂度的各种组件。

## 5.1.2 顺序块

现在我们可以更仔细地看看Sequential类是如何工作的，回想一下Sequential的设计是为了把其他模块串起来。为了构建我们自己的简化的MySequential，我们只需要定义两个关键函数：
1.一个将块逐个追加到列表中的函数；
2.一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”

In [4]:
# MySequential类提供了与默认Sequential类相同的功能
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中_module的类型是OrderedDict
            self._modules[str(idx)] = module
            
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0320, -0.1269,  0.0310, -0.0151,  0.0123,  0.1173,  0.3469, -0.1395,
         -0.0105,  0.2393],
        [ 0.0669, -0.0367,  0.0934,  0.1619, -0.0846,  0.1724,  0.3466, -0.0469,
         -0.0725,  0.2247]], grad_fn=<AddmmBackward0>)

## 5.1.3 在前向传播函数中执行代码

Sequential类使模型构造变得简单，允许我们组合新的架构，而不必定义自己的类。然而，并不是所有架构都是简单的顺序架构。

到目前为止，我们网络中的所有操作都对网络的激活值及网络参数起作用。然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，我们称之为常数参数。

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连结层。这相当于两个全连接层的共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层。其权重在实例化时被随机初始化，之后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。然后，神经网络将这个固定层的输出通过一个全连接层。

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0995, grad_fn=<SumBackward0>)

可以混合搭配各种组合块。

In [8]:
# 嵌套块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0942, grad_fn=<SumBackward0>)

## 5.1.4 效率

## 5.1.5 小结

1.一个块可以有多个层组成；一个块可以由许多块组成；  
2.块可以包含代码；  
3.块负责大量的内部处理，包含参数初始化和反向传播；  
4.层和块的顺序连接由Sequential块处理。